In [3]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2022"), day=int("17"))

puzzle.header()

# Pyroclastic Flow

[Open Website](https://adventofcode.com/2022/day/17)

In [34]:
from common.matrix import Direction


# helper functions
class Shape:
    def __init__(self, shape: list):
        self.shape = shape
        self.width = len(shape[0])
        self.height = len(shape)
        self.points = []
        for y, row in enumerate(shape):
            for x, cell in enumerate(row):
                if cell == "#":
                    self.points.append([x, y])

    def __str__(self):
        return "\n".join(self.shape)

    def __iter__(self):
        return iter(self.points)

    def copy(self):
        return Shape(self.shape)

    def collides(self, shapes: list) -> bool:
        for shape in shapes:
            for point in self.points:
                if point in shape.points:
                    return True
        return False

    def set_floor(self, floor: int):
        # move up to floor
        for point in self.points:
            point[1] += floor

    def move(self, direction: Direction, shapes: list, max_x: int) -> bool:
        # revert function
        def revert():
            for point in self.points:
                if direction == Direction.DOWN:
                    point[1] += 1
                elif direction == Direction.UP:
                    point[1] -= 1
                elif direction == Direction.LEFT:
                    point[0] += 1
                elif direction == Direction.RIGHT:
                    point[0] -= 1

        # move
        for point in self.points:
            if direction == Direction.DOWN:
                point[1] -= 1
            elif direction == Direction.UP:
                point[1] += 1
            elif direction == Direction.LEFT:
                point[0] -= 1
            elif direction == Direction.RIGHT:
                point[0] += 1

        # check if out of bounds
        for point in self.points:
            if point[0] < 0 or point[0] >= max_x:
                revert()
                return False

        # check if y is out of bounds
        if direction == Direction.DOWN:
            for point in self.points:
                if point[1] < 0:
                    revert()
                    return False

        # if collides, revert
        if self.collides(shapes):
            revert()
            return False

        return True


def domain_from_input(input: InputReader) -> (list, list):
    shapes = [[
        list("####")
    ], [
        list(".#."),
        list("###"),
        list(".#."),
    ], [
        list("###"),
        list("..#"),
        list("..#"),
    ], [
        list("#"),
        list("#"),
        list("#"),
        list("#")
    ], [
        list("##"),
        list("##")
    ]]
    directions = list(input.as_str())
    return directions, shapes


test_input, test_shapes = domain_from_input(puzzle.example(0))
print(test_input)
for shape in test_shapes:
    print(shape)

['>', '>', '>', '<', '<', '>', '<', '>', '>', '<', '<', '<', '>', '>', '<', '>', '>', '>', '<', '<', '<', '>', '>', '>', '<', '<', '<', '>', '<', '<', '<', '>', '>', '<', '>', '>', '<', '<', '>', '>']
[['#', '#', '#', '#']]
[['.', '#', '.'], ['#', '#', '#'], ['.', '#', '.']]
[['#', '#', '#'], ['.', '.', '#'], ['.', '.', '#']]
[['#'], ['#'], ['#'], ['#']]
[['#', '#'], ['#', '#']]


In [36]:
# test case (part 1)
def part_1(reader: InputReader, max_shapes: int, debug: bool) -> int:
    directions, shapes = domain_from_input(reader)
    if debug:
        print(directions)

    max_x = 7

    def next_shape():
        shape = shapes[len(history) % len(shapes)]
        shape = Shape(shape)
        max_y = find_max_y()
        shape.set_floor(max_y + 3)
        shape.move(Direction.RIGHT, [], max_x)
        shape.move(Direction.RIGHT, [], max_x)
        return shape

    def find_max_y():
        max_y = 0
        for shape in history:
            for point in shape:
                max_y = max(max_y, point[1] + 1)
        return max_y

    def print_shapes():
        max_y = find_max_y()
        for point in current_shape:
            max_y = max(max_y, point[1])

        for y in reversed(range(max_y + 2)):
            row = []
            for x in range(max_x):
                row.append(".")
            for shape in history:
                for point in shape:
                    if point[1] == y:
                        row[point[0]] = "#"

            for point in current_shape:
                if point[1] == y:
                    row[point[0]] = "@"

            print("".join(row))
        print()

    history = []
    current_shape = next_shape()
    direction_counter = 0

    while len(history) < max_shapes:
        next_direction = directions[direction_counter % len(directions)]
        direction_counter += 1

        if debug:
            print(f"Direction: {next_direction}")
            print_shapes()

        # Move direction
        if next_direction == ">":
            current_shape.move(Direction.RIGHT, history, max_x)
        elif next_direction == "<":
            current_shape.move(Direction.LEFT, history, max_x)

        if debug:
            print(f"Direction: v")
            print_shapes()

        # Move down
        ok = current_shape.move(Direction.DOWN, history, max_x)

        if not ok:
            history.append(current_shape)
            current_shape = next_shape()

    if debug:
        print("Final")
        print_shapes()

    return find_max_y()


result = part_1(puzzle.example(0), 2022, False)
print(result)
assert result == 3068

3068


In [37]:
# real case (part 1)
result = part_1(puzzle.input(), 2022, False)
print(result)

3083


In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        print(lines)
    return 0


result = part_2(puzzle.example(0), True)
print(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
print(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()